In [1]:
#Bibliotecas a importar
import numpy as np
import math as math
import itertools

### Valuación de Opciones en tiempo discreto

In [10]:
#t_op: Tipo de opcion
# 1: Opción europea tipo call
# 2: Opción europea tipo put

#No hagan caso a estos
# 3: Opción digital tipo call
# 4: Opción digital tipo put
# 5: Opción asiática
# 6: Opción lookback
# 7: Opción Barrera Up&Out
# 8: Opción Barrera Up&In
# 9: Opción Americana call
# 10: Opción Americana put

t_op = 1
T = 1 #Tiempo de vigencia de la opción (en años)
r = 0.056 #tasa anual libre de riesgo continua
k = 4 #Precio strike. Funciona como el salto en el caso de las digitales
S_0 = 5 #Precio del subyacente a t=0
n = 3 #Particiones del árbol (Periodos)
sigma= 0 #Volatilidad. Si es diferente a 0 calculamos u y d como en el caso contínuo
u = 1.2 #Factor de aumento en el precio
d = 0.6 #Factor de disminución de precio

#No hagan caso a estos aún, son de otros derivados
reb = 2 #Rebate
b = 5 #Barrera

# Probabilidad en la medida libre de riesgo

En la medida libre de riesgo, calculamos las probabilidades de subida y bajada de la sguiente manera:

$$P_{u}=\frac{e^{r*\delta_t}-d}{u-d}$$
$$P_{d}=\frac{u-e^{r*\delta_t}}{u-d}$$

Donde $\delta_n$ es la proporción de años que dura cada periodo en el que particionamos el árbol

In [11]:
delta_t = T / n
if (sigma!=0):
    u=math.exp((r-(sigma**2)/(2))*delta_t+sigma*math.sqrt(delta_t))
    d=math.exp((r-(sigma**2)/(2))*delta_t-sigma*math.sqrt(delta_t))
p = (math.exp(r * delta_t) - d) / (u - d) #Probabilidad de aumento en el precio

## Construcción del árbol

### Subyacente

Valor en el primer paso: Es colocar $S_0$ en la entrada $[0,0]$ de la matriz `arbol`, la cual contendrá en sus entradas [i,j] los posibles precios que el subyacente podrá tomar después de `i` bajadas en su precio, al tiempo `j`

In [12]:
#Matriz que almacena los valores del subyacente
arbol = np.zeros((n + 1, n + 1))
#El primer valor de la matriz es el subyacente a precio en t=0
arbol[0,0] = S_0
arbol #Visualizamos

array([[5., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

Como asumimos un árbol recombinante, al tiempo `j`, habrá a lo más `j+1` posibles precios del activo subyacente, el cual estará determinado por:
$$S_j=S_0*d(i)*u(j-i)$$

* $S_j$ es el precio del activo subyacente a tiempo j
* $d$ el factor de bajada
* $u$ el factor de subida
* $i$ el número de bajadas que ha sufrido el precio del activo subyacente hasta el tiempo $j$

In [13]:
#For para llenar los valores de la matriz
for col in range(1, n +1):
    for ren in range(0, n +1):
        #Condicional para limitar matriz superior triangular
        if((col - ren) >= 0):
            arbol[ren, col] = S_0 *(( u ** (col - ren)) * (d ** (ren)))
arbol #Visualizamos

array([[5.  , 6.  , 7.2 , 8.64],
       [0.  , 3.  , 3.6 , 4.32],
       [0.  , 0.  , 1.8 , 2.16],
       [0.  , 0.  , 0.  , 1.08]])

No hagan caso a esto aún, es extra :)

In [72]:
if(t_op==5 or t_op==6 or t_op==7 or t_op==8):
    arbol_aux=np.zeros((2**n, n+1))
    for col in range(0,n+1):
        if(col==0):
            arbol_aux[0,0]=S_0
        else:
            up_down=list(itertools.product(['u','d'],repeat=col))
            for ren in range(0,2**col):
                bajas=0
                pos=up_down[ren]
                for letter in pos:
                    if (letter=='d'):
                        bajas=bajas+1
                arbol_aux[ren,col]=S_0*(u**(col-bajas))*d**(bajas)

### Payoff

Como valuaremos utilizando una metodología backwards nuestro árbol, tenemos que comenzar viendo el payoff del tiempo de vencimiento. Es decir, en una matriz llamada `payoff`, del mismo tamaño que la matriz `arbol`, vamos a poner en la entrada [i,n] (es decir, a tiempo de vencimiento), el payoff correspondiente.

En el caso del call:
$$Payoff_{[i,n]}=(S_{[i,n]}-k)_{+}$$

En el caso del put:
$$Payoff_{[i,n]}=(k-S_{[i,n]})_{+}$$

In [14]:
#Matriz que almacena el valor de los payoffs
payoffs = np.zeros((n+1, n+1))
#Sobreescribimos la última columna de la matriz de PAYOFSS
#IMPORTANTE: Agregar condicional para el caso del put (este es el caso particular del call)
if (t_op==1 or t_op==9):
    for ren in range(0, n+1):
        payoffs[ren, n] = max(arbol[ren, n] - k, 0)
elif (t_op==2 or t_op==10):
    for ren in range(0, n+1):
        payoffs[ren, n] = max(k-arbol[ren, n] , 0)
elif t_op==3:
    for ren in range(0, n+1):
        if (arbol[ren, n] > S_0):
            payoffs[ren, n] = k
        else:
            payoffs[ren, n] = 0
elif t_op==4:
    for ren in range(0, n+1):
        if (arbol[ren, n] < S_0):
            payoffs[ren, n] = k
        else:
            payoffs[ren, n] = 0
elif t_op==5:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        suma=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            suma=suma+arbol[bajas,(i+1)]
        payoffs[ren,n]=max((suma/(n+1))-k,0)
elif t_op==6:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        valores=np.zeros(n+1)
        valores[0]=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            valores[i+1]=arbol[bajas,(i+1)]
        payoffs[ren,n]=max(valores)
elif t_op==7:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        valores=np.zeros(n+1)
        valores[0]=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            valores[i+1]=arbol[bajas,(i+1)]
        if(max(valores)>=b):
            payoffs[ren,n]=reb
        else:
            payoffs[ren,n]=max(arbol_aux[ren, n] - k, 0)
elif t_op==8:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        valores=np.zeros(n+1)
        valores[0]=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            valores[i+1]=arbol[bajas,(i+1)]
        if(max(valores)<=b):
            payoffs[ren,n]=reb
        else:
            payoffs[ren,n]=max(k-arbol_aux[ren, n], 0)
payoffs #Visualizamos

array([[0.  , 0.  , 0.  , 4.64],
       [0.  , 0.  , 0.  , 0.32],
       [0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  ]])

Valuamos backwards, al traer a valor presente, el valor esperado del derivado, es decir, si $X$ es la variable aleatoria del valor presente del payoff del derivado, buscamos:

$$\text{Valor del derivado a tiempo $t$}=\mathbb{E}_{\mathbb{Q}}[X|\mathcal{F}_{t}]$$

Es decir:

$$\text{Valor del derivado a tiempo $t$}=e^{-r*\delta_{t}}\left[p\left(X_u\right) + \left(1-p\right)\left(X_d\right) \right]$$

Donde:

* X_u es el valor del payoff al tiempo $t+1$, suponiendo que el valor del activo subyacente sube
* X_d es el valor del payoff al tiempo $t+1$, suponiendo que el valor del activo subyacente baja
* p * X_u es la probabilidad de que el valor del activo subyacente suba
* r es la tasa de interés libre de riesgo (Supuesta continua aquí)
* \delta_t es la proporción de años que hay en cada intervalo de tiempo en el que particionamos nuestro ñarbol

In [15]:
#Sobreescribimos matriz de payoffs mediante el método iterativo
if(t_op!=5):
    for i in range(1, n + 1):
        col = n - i
        for j in range(0, col +1):
            payoffs[j, col] = math.exp(-r * delta_t) * (p * payoffs[j, col + 1] + (1 - p)*payoffs[j +1, col +1])

if(t_op==5 or t_op==6 or t_op==7 or t_op==8):
    for i in range(1, n+1):
        col = n - i
        for j in range(0, 2**col):
            payoffs[j, col] = math.exp(-r * delta_t) * (p * payoffs[2*j, col + 1] + (1 - p)*payoffs[2*j +1, col +1])
if(t_op==9):
    for i in range(1, n + 1):
        col = n - i
        for j in range(0, col +1):
            payoffs[j, col] = max(arbol[j,col]-k,math.exp(-r * delta_t) * (p * payoffs[j, col + 1] + (1 - p)*payoffs[j +1, col +1]))
if(t_op==10):
    for i in range(1, n + 1):
        col = n - i
        for j in range(0, col +1):
            payoffs[j, col] = max(k-arbol[j,col],math.exp(-r * delta_t) * (p * payoffs[j, col + 1] + (1 - p)*payoffs[j +1, col +1]))
payoffs #visualizamos

array([[1.62598469, 2.30817101, 3.27397409, 4.64      ],
       [0.        , 0.15022224, 0.21925126, 0.32      ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

## Valor de la opción

Estará determinado por la entrada [0,0] de la matriz `payoff`

In [16]:
payoffs[0,0]

1.6259846893541077

## Composición del portafolio

# Alfas

$$\alpha = \frac{X_u-X_d}{S_u-S_d}$$

In [8]:
#Matriz que almacena los valores de las alfas
alfas = np.zeros((n, n ))
if(t_op!=5):
    for ren in range(0,n):
        for col in range(0, n):
            #Condicional para limitar matriz superior triangular
            if((col - ren) >= 0):
                alfas[ren,col]=(payoffs[ren,col+1]-payoffs[ren+1,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1])
if(t_op==5 or t_op==6 or t_op==7 or t_op==8):
    alfas = np.zeros((2**n, n))
    for col in range(0,n):
    #Condicional para limitar matriz superior triangular
        for ren in range(0, 2**col):
            alfas[ren,col]=(payoffs[2*ren,col+1]-payoffs[2*ren+1,col+1])/(arbol_aux[2*ren,col+1]-arbol_aux[2*ren+1,col+1])
alfas #Visualizamos

array([[0.25662965, 0.31212853, 0.37962963],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

# Betas

$$\beta = e^{-r*\delta_t}\left(X_u-\frac{(X_u-X_d)S_u}{S_u-S_d}\right)$$

In [9]:
#Matriz que almacena los valores de las alfas
if(t_op!=5):
    betas = np.zeros((n, n ))
    for ren in range(0,n):
        for col in range(0, n):
            #Condicional para limitar matriz superior triangular
            if((col - ren) >= 0):
                betas[ren,col]=math.exp(-r * delta_t)*(payoffs[ren,col+1]-((payoffs[ren,col+1]-payoffs[ren+1,col+1])*arbol[ren,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1]))

if(t_op==5 or t_op==6 or t_op==7 or t_op==8):
    betas = np.zeros((2**n, n))
    for col in range(0,n):
    #Condicional para limitar matriz superior triangular
        for ren in range(0, 2**col):
                betas[ren,col]=math.exp(-r * delta_t)*(payoffs[2*ren,col+1]-((payoffs[2*ren,col+1]-payoffs[2*ren+1,col+1])*arbol_aux[2*ren,col+1])/(arbol_aux[2*ren,col+1]-arbol_aux[2*ren+1,col+1]))
betas #Visualizamos

array([[-0.755651  , -1.10288223, -1.60967062],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

![¡Bonito lunes a tod@s!](Snoopy.jpg)